In [ ]:
c

coordsbbox empty


In [6]:
urls = ["www.362_412.shp", "www.365_412.shp"]
match_string = [[], "shp"]

for url in urls:
    if all(substring in url for substring in match_string if substring != []):
        print(f"Found URL with both '365_412' and 'shp': {url}")
        break
else:
    print("No URL found with both '365_412' and 'shp'")

Found URL with both '365_412' and 'shp': www.362_412.shp


In [77]:
import laspy
from pathlib import Path
import numpy as np
from scipy import stats
import pandas as pd
import glob

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Dataset

In [88]:
pd.Series(d_z).describe()

In [ ]:
sites = ['SJER', 'SOAP', 'TEAK']
data_ls = []

for site in sites:
    data_globs = glob.glob('../data/processed/{}/20*/output'.format(site))

    for data_glob in data_globs:
        data_path = Path(data_glob)
        laz_files = [i for i in data_path.glob('*.laz')]
        indexes = []
        point_data_z = []

        for laz_file in laz_files:
            plot_sub_id = laz_file.stem
            indexes.append(plot_sub_id)
            data = laspy.read(laz_file)
            point_data = np.stack([data.X, data.Y, data.Z], axis=0).transpose((1, 0)) * data.header.z_scale + data.header.z_offset
            # mask = pd.Series(data.classification.array).isin([2,3,4,5])
            # point_data = point_data[mask]
            d_z = point_data[:, 2]
            d = pd.Series(d_z).describe()
            d['skew'] = stats.skew(d_z)
            d['kurtosis'] = stats.kurtosis(d_z)
            point_data_z.append(d)
            # add year and subplotID
        
        point_data_z_df = pd.DataFrame(point_data_z)
        point_data_z_df['plot_sub_id'] = indexes
        
        bio_df = pd.read_csv('{}/plot_level_pp_veg_structure_IND_IBA_IAGB_live.csv'.format(data_glob))
        bio_df['plot_sub_id'] = bio_df['plotID'].astype(str) + '_' + bio_df['subplotID'].astype(str)
        data_df_temp = point_data_z_df.set_index('plot_sub_id').join(bio_df.set_index('plot_sub_id')).dropna()
        data_ls.append(data_df_temp)

data_df = pd.concat(data_ls) 
data_df  

# Train linear regression

In [136]:
X = data_df[['mean', 'std', 'skew', 'kurtosis']].values
y = data_df.biomass

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2)
model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

train_score = model.score(X_train, y_train)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')
print(f'R²: {train_score:.2f}')

Mean absolute error: 42.81
Mean squared error: 2102.37
Root mean squared error: 45.85
R²: 0.05


In [135]:
pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Diff': abs(y_test-y_pred)})

,Actual,Predicted,Diff
plot_sub_id,,,
TEAK_062_21,0.000000,20.925920,20.925920
SOAP_016_central,0.000417,51.067154,51.066737
TEAK_059_41,0.000000,38.237595,38.237595
SOAP_008_central,7.192184,75.940845,68.748661
SJER_008_central,0.025419,10.502814,10.477394
TEAK_045_39,5.897503,19.576834,13.679331
TEAK_054_41,0.000000,9.662002,9.662002
SOAP_018_central,7.227835,29.602295,22.374460
TEAK_053_21,0.000000,48.782161,48.782161
